
# ANIME RECOMMENDER SYSTEM


In [10]:
# Importation of required libraries

import json
import pandas as pd
import brotli

from sklearn.preprocessing import MinMaxScaler

In [11]:
pd.set_option('display.max_columns',500)

In [17]:
df = pd.read_csv('./train_data/anime.csv')
df

,Anime Title,MAL Url,English,Japanese,Type,Episodes,Status,Aired,Premiered,Broadcast,Producers,Licensors,Studios,Source,Genres,Duration,Rating,Score,Ranking,Popularity,Members,Favorites,Summary
0,Fullmetal Alchemist: Brotherhood,https://myanimelist.net/anime/5114/Fullmetal_A...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,TV,64,Finished Airing,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,Sundays at 17:00 (JST),"Aniplex, Square Enix, Mainichi Broadcasting Sy...","Funimation, Aniplex of America",Bones,Manga,"Action, Military, Adventure, Comedy, Drama, Ma...",24 min. per ep.,R - 17+ (violence & profanity),9.22,1,4,2025613,167812,"""In order for something to be obtained, someth..."
1,Steins;Gate,https://myanimelist.net/anime/9253/Steins_Gate,Steins;Gate,STEINS;GATE,TV,24,Finished Airing,"Apr 6, 2011 to Sep 14, 2011",Spring 2011,Wednesdays at 02:05 (JST),"Frontier Works, Media Factory, Movic, AT-X, Ka...",Funimation,White Fox,Visual novel,"Thriller, Sci-Fi",24 min. per ep.,PG-13 - Teens 13 or older,9.12,2,8,1636151,139909,The self-proclaimed mad scientist Rintarou Oka...
2,Gintama°,https://myanimelist.net/anime/28977/Gintama°,Gintama Season 4,銀魂°,TV,51,Finished Airing,"Apr 8, 2015 to Mar 30, 2016",Spring 2015,Wednesdays at 18:00 (JST),"TV Tokyo, Aniplex, Dentsu","Funimation, Crunchyroll",Bandai Namco Pictures,Manga,"Action, Comedy, Historical, Parody, Samurai, S...",24 min. per ep.,PG-13 - Teens 13 or older,9.11,3,330,363879,10353,"Gintoki, Shinpachi, and Kagura return as the f..."
3,Hunter x Hunter (2011),https://myanimelist.net/anime/11061/Hunter_x_H...,Hunter x Hunter,HUNTER×HUNTER（ハンター×ハンター）,TV,148,Finished Airing,"Oct 2, 2011 to Sep 24, 2014",Fall 2011,Sundays at 10:55 (JST),"VAP, Nippon Television Network, Shueisha",Viz Media,Madhouse,Manga,"Action, Adventure, Fantasy, Shounen, Super Power",23 min. per ep.,PG-13 - Teens 13 or older,9.11,4,14,1459320,130492,Hunter x Hunter is set in a world where Hunter...
4,Ginga Eiyuu Densetsu,https://myanimelist.net/anime/820/Ginga_Eiyuu_...,Legend of the Galactic Heroes,銀河英雄伝説,OVA,110,Finished Airing,"Jan 8, 1988 to Mar 17, 1997",NaN,NaN,"Kitty Films, K-Factory",Sentai Filmworks,"Artland, Magic Bus",Novel,"Military, Sci-Fi, Space, Drama",26 min. per ep.,R - 17+ (violence & profanity),9.10,5,629,212162,13100,The 150-year-long stalemate between the two in...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Roba Chotto Suneta,https://myanimelist.net/anime/30076/Roba_Chott...,NaN,ロバちょっとすねた,Music,1,Finished Airing,"Apr 3, 1983",NaN,NaN,NHK,NaN,NaN,Original,"Kids, Music",2 min.,G - All Ages,5.24,9997,13717,255,0,A music video for the song by Agnes Chan that ...
9996,Ski Jumping Pairs: Road to Torino 2006,https://myanimelist.net/anime/10627/Ski_Jumpin...,NaN,スキージャンプ・ペア ~Road to TORINO 2006~,Movie,1,Finished Airing,"Jan 28, 2006",NaN,NaN,NaN,NaN,NaN,Unknown,"Comedy, Sports",4 min.,G - All Ages,5.24,9998,12571,370,1,A CG animated film by Riichiro Mashima.\n\r\nA...
9997,Tentoumushi no Otomurai,https://myanimelist.net/anime/12521/Tentoumush...,Ladybirds' Requiem,てんとう虫のおとむらい,Movie,1,Finished Airing,2006,NaN,NaN,NaN,NaN,NaN,Original,"Music, Dementia",5 min.,G - All Ages,5.24,9999,11865,494,2,"The story of the animation begins from a girl,..."
9998,The Baby Birds of Norman McLaren,https://myanimelist.net/anime/28765/The_Baby_B...,NaN,The Baby Birds of Norman McLaren,Movie,1,Finished Airing,2014,NaN,NaN,Mirai Film,NaN,NaN,Unknown,Music,3 min.,G - All Ages,5.24,10000,12098,452,0,Celebrating the 100th year of birth of Norman ...


In [14]:
def replace_bottom(word):
        word = word.replace('[Written by MAL Rewrite]', '')
        return word
    
class AnimeRecom:
    
    def __init__(self):
        self.anime = './train_data/anime.csv'
        self.scaler = MinMaxScaler()
        
    def read_files(self):
        df = pd.read_csv(self.anime)
        return df
    
    

    def process_files(self):
        
        # retrieve the anime dataset from the read_files function
        df = self.read_files()
        
        # Reshape the values to be consistent between 0 and 1
        df = df.assign(
            Score=self.scaler.fit_transform(
                df.Score.values.reshape(-1, 1)
            )
        )
        
        # Select a subset of the attributes
        to_search = df[['Rating', 'Summary', 'Genres', 'Anime Title', 'English', 'MAL Url']]
        
        
        # Resetting the index and dropping rows with missing values
        to_search = to_search.reset_index()
        to_search.drop_duplicates(inplace=True)
        to_search.reset_index(inplace=True)
        to_search.drop('index', axis=1, inplace=True)
        to_search.to_parquet('./data/anime/animes.parquet', compression='brotli')
        return to_search
    
    def column_combination(self):
        to_search = self.process_files()
        
        # Joining the three desired columns together
        to_search['genre'] = to_search['Rating']  + "\n" + to_search['Genres'] + '\n' + to_search['Summary']
        
        # Create a new column for retrieval later on
        to_search['feature'] = to_search['Rating']  + "\n" + to_search['Anime Title'] + "\n" + to_search['Genres'] + '\n' + to_search['Summary']
        
        # Convert all to string
        to_search['genre'] = to_search['genre'].astype(str)
        
        # Apply the replace bottom to the genre function
        to_search['genre'] = to_search['genre'].apply(replace_bottom)
        to_search.drop('level_0', inplace=True, axis=1)
        return to_search
    
    def save_index(self, search):
        search.reset_index(inplace=True)
        movies_index = search['index'].tolist()
        dict_bytes = str.encode(str(movies_index))
        compressed = brotli.compress(dict_bytes)

        with open('./data/anime/id.brotli', 'wb') as f:
            f.write(compressed)
            
    def save_embeddings(self, search):
        model = SentenceTransformer('stsb-roberta-large')
        embeddings = model.encode(search['genre'].tolist(), convert_to_tensor=True)
        torch.save(embeddings, './data/anime/an_combined_embeddings.pt')
if __name__ == '__main__':
    anim = AnimeRecom()
    df = anim.column_combination()
    anim.save_index(df)
    anim.save_embeddings(df)

In [15]:
df

,index,Rating,Summary,Genres,Anime Title,English,MAL Url,genre,feature
0,0,R - 17+ (violence & profanity),"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...",Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: Brotherhood,https://myanimelist.net/anime/5114/Fullmetal_A...,"R - 17+ (violence & profanity)\nAction, Milita...",R - 17+ (violence & profanity)\nFullmetal Alch...
1,1,PG-13 - Teens 13 or older,The self-proclaimed mad scientist Rintarou Oka...,"Thriller, Sci-Fi",Steins;Gate,Steins;Gate,https://myanimelist.net/anime/9253/Steins_Gate,"PG-13 - Teens 13 or older\nThriller, Sci-Fi\nT...",PG-13 - Teens 13 or older\nSteins;Gate\nThrill...
2,2,PG-13 - Teens 13 or older,"Gintoki, Shinpachi, and Kagura return as the f...","Action, Comedy, Historical, Parody, Samurai, S...",Gintama°,Gintama Season 4,https://myanimelist.net/anime/28977/Gintama°,"PG-13 - Teens 13 or older\nAction, Comedy, His...","PG-13 - Teens 13 or older\nGintama°\nAction, C..."
3,3,PG-13 - Teens 13 or older,Hunter x Hunter is set in a world where Hunter...,"Action, Adventure, Fantasy, Shounen, Super Power",Hunter x Hunter (2011),Hunter x Hunter,https://myanimelist.net/anime/11061/Hunter_x_H...,"PG-13 - Teens 13 or older\nAction, Adventure, ...",PG-13 - Teens 13 or older\nHunter x Hunter (20...
4,4,R - 17+ (violence & profanity),The 150-year-long stalemate between the two in...,"Military, Sci-Fi, Space, Drama",Ginga Eiyuu Densetsu,Legend of the Galactic Heroes,https://myanimelist.net/anime/820/Ginga_Eiyuu_...,"R - 17+ (violence & profanity)\nMilitary, Sci-...",R - 17+ (violence & profanity)\nGinga Eiyuu De...
...,...,...,...,...,...,...,...,...,...
9995,9995,G - All Ages,A music video for the song by Agnes Chan that ...,"Kids, Music",Roba Chotto Suneta,NaN,https://myanimelist.net/anime/30076/Roba_Chott...,"G - All Ages\nKids, Music\nA music video for t...","G - All Ages\nRoba Chotto Suneta\nKids, Music\..."
9996,9996,G - All Ages,A CG animated film by Riichiro Mashima.\n\r\nA...,"Comedy, Sports",Ski Jumping Pairs: Road to Torino 2006,NaN,https://myanimelist.net/anime/10627/Ski_Jumpin...,"G - All Ages\nComedy, Sports\nA CG animated fi...",G - All Ages\nSki Jumping Pairs: Road to Torin...
9997,9997,G - All Ages,"The story of the animation begins from a girl,...","Music, Dementia",Tentoumushi no Otomurai,Ladybirds' Requiem,https://myanimelist.net/anime/12521/Tentoumush...,"G - All Ages\nMusic, Dementia\nThe story of th...","G - All Ages\nTentoumushi no Otomurai\nMusic, ..."
9998,9998,G - All Ages,Celebrating the 100th year of birth of Norman ...,Music,The Baby Birds of Norman McLaren,NaN,https://myanimelist.net/anime/28765/The_Baby_B...,G - All Ages\nMusic\nCelebrating the 100th yea...,G - All Ages\nThe Baby Birds of Norman McLaren...


## SENTENCE TRANSFORMERS

In [4]:
from sentence_transformers import SentenceTransformer



torch.save(an_combined_embeddings, './data/anime/an_combined_embeddings.pt')

In [ ]:
torch.get_num_threads()